In [1]:
import pickle
import ROOT
import numpy as np
import pandas as pd

from ROOT import TH1F, TCanvas

Welcome to JupyROOT 6.28/09


In [2]:
with open("results.root", "rb") as f:
    data = pickle.load(f)

In [3]:
cos_theta_true_L = data["cos_theta_true_L"]
cos_theta_pred_L = data["cos_theta_pred_L"]
cos_theta_true_R = data["cos_theta_true_R"]
cos_theta_pred_R = data["cos_theta_pred_R"]

In [4]:
def fit_beta(angles, title, polarization, results):
    # Convert list to numpy array
    angles = np.array(angles)
    
    # Define polarization
    if "L" in polarization:
        p=-1
    if "R" in polarization:
        p=1    
    
    # Book Histogram
    hist_angles = TH1F( title, title, 100, -1, 1 )

    # Book Canvas
    c = ROOT.TCanvas("c", "Histogram Canvas", 800, 600)
    
    # Fill Histogram
    for angle in angles:
        hist_angles.Fill(angle)
        
    # Draw Histogram
    hist_angles.Draw()
    #c.SaveAs(title+".png")
    #c.Draw()
        
    # Define Custom Fit Function
    def beta_function(costheta, parameters):
        return 1/2*(1+parameters[0]*costheta[0])
    
    # Initialize Fit Function
    fit = ROOT.TF1("fit", beta_function, -1, 1, 1)
    fit.SetParameters(0)
    fit.SetParNames("p*Beta")
    
    # Perform Fit
    hist_angles.Fit(fit, "q")
    
    # Print Results
    fitted_beta = fit.GetParameters()[0]*p
    method_2 = np.mean(angles)*3*p
    positive_angles = len(angles[angles>0])
    negative_angles = len(angles[angles<0])
    method_3 = (positive_angles-negative_angles)/(positive_angles+negative_angles)*2*p
        
    new_row = pd.DataFrame([{'method1': fitted_beta, 'method2': method_2, 'method3': method_3}], index=[title])

    df = pd.concat([results, new_row])
    
    return df

In [5]:
results_DF = pd.DataFrame({'method1': [],
                         'method2': [],
                         'method3': [],
                        })

In [6]:
results_DF = fit_beta(cos_theta_true_L, "true_L", "L", results_DF)
results_DF = fit_beta(cos_theta_pred_L, "pred_L", "L", results_DF)
results_DF = fit_beta(cos_theta_true_R, "true_R", "R", results_DF)
results_DF = fit_beta(cos_theta_pred_R, "pred_R", "R", results_DF)

In [7]:
print(results_DF)

         method1   method2   method3
true_L  0.993185  1.011050  0.951820
pred_L  0.674366  0.722116  0.704289
true_R  1.011147  1.073316  1.096108
pred_R  0.640584  0.667085  0.656086
